 use neural network to predict the change in the ecmwf-eps from the previous cycle run 12 hours ago for days 9-14
 ### current model features
- the difference between the new ecmwf forecast for day 9 and the previous emcwf-eps forecast for day 9[x]
- the difference between the new emcwf-eps forecast for day 8 and the previous emcwf-eps forecast for day 8[x]
- the difference between the new gfs-ens-bc forecast for days 9-14 and the previous emcwf-eps forecast for days 9-14[x]
- the difference between the new cmc-ens forecast for days 9-14 and the new gfs-ens-bc  forecast for days 9-14[x]

### new feature ideas
- rain data
- wind data
- ???

In [53]:
import pandas as pd
import glob
import matplotlib.pyplot as plt

In [54]:
path = "RawData"

In [303]:
def extract_date_time(filename):
    parts = filename.split('.')
    date = parts[1]
    time = parts[2]
    return date, time

In [ ]:
ecmwf_files = glob.glob(path + '/ecmwf.*.[01][02].gw_hdd.csv')
ecmwf_sorted_files = sorted(ecmwf_files, key=lambda x: (x.split('.')[1], x.split('.')[2]))[3:]

emcwf_eps_files = glob.glob(path + '/ecmwf-eps.*.[01][02].gw_hdd.csv')
emcwf_eps_sorted_files = sorted(emcwf_eps_files, key=lambda x: (x.split('.')[1], x.split('.')[2]))[2:]

In [ ]:
set1 = set((extract_date_time(filename) for filename in ecmwf_sorted_files))
set2 = set((extract_date_time(filename) for filename in emcwf_eps_sorted_files))

In [ ]:
ecmwf_sorted_files = [filename for filename in ecmwf_sorted_files if extract_date_time(filename) in set2]
emcwf_eps_sorted_files = [filename for filename in emcwf_eps_sorted_files if extract_date_time(filename) in set1]

### step 1 get changes in ecmwf-eps compared to 12 hours ago for days 9-14

In [331]:
change_df = pd.DataFrame(columns=['time', 'diff_9', 'diff_10', 'diff_11', 'diff_12', 'diff_13', 'diff_14'])
for i in range(1, len(emcwf_eps_sorted_files)):
    emcwf_eps_df = pd.read_csv(emcwf_eps_sorted_files[i])
    prev_emcwf_eps_df = pd.read_csv(emcwf_eps_sorted_files[i-1])
    date = emcwf_eps_df[emcwf_eps_df.iloc[:, 2] == 0].iloc[-1]['Date']
    time = int(emcwf_eps_sorted_files[i-1].split('.')[2])
    changes = [time]
    for day in range(8, 14):
        changes.append(emcwf_eps_df.iloc[day]['Value'] - prev_emcwf_eps_df.iloc[day]['Value'])
    new_row = pd.DataFrame([changes], columns=change_df.columns, index=[date])
    change_df = pd.concat([change_df, new_row])


## step 2 recreate model

### step 2.1 get difference between the new ecmwf forecast for day 9 and the previous emcwf-eps forecast for day 9

### problems:
- not all dates are in both sets of data

In [275]:
diff = {}
for i in range(2, len(ecmwf_sorted_files)-1):
    ecmwf_df = pd.read_csv(ecmwf_sorted_files[i])
    emcwf_eps_df = pd.read_csv(emcwf_eps_sorted_files[i-1])

    ecmwf = ecmwf_df.iloc[8]
    emcwf_eps = emcwf_eps_df.iloc[8]

    date = ecmwf['Date']
    diff[date] = ecmwf['Value'] - emcwf_eps['Value']

In [277]:
diff_df = pd.DataFrame.from_dict(diff, orient='index', columns=['Value'])

In [305]:
diff_df

,Value
2018-07-12,0.002
2018-07-13,0.000
2018-07-14,0.003
2018-07-15,0.001
2018-07-16,0.003
...,...
2023-05-13,-0.277
2023-05-14,-0.284
2023-05-15,-0.412
2023-05-16,-0.251
